In [ ]:
%matplotlib inline
import torch
from sebm.sgld import SGLD_sampler
from sebm.cebm_sgld import init_cebm
dataset =  'cifar10' # 'svhn' # 'cifar10' # 'mnist' #  'flowers102' #
if dataset == 'mnist' or dataset =='fashionmnist':
    input_channels, im_height, im_width = 1, 28, 28
else:
    input_channels, im_height, im_width = 3, 32, 32
device = torch.device('cuda:1')
arch =  'simplenet2' # 'wresnet' # 'simplenet'
ss = 2
seed = 123

lr, reg_alpha =  1e-4, 1e-1
optimize_priors = False
if dataset == 'cifar10' or dataset == 'svhn':
    channels, kernels, strides, paddings =[64,128,256,512], [3,4,4,4], [1,2,2,2], [1,1,1,1]
    latent_dim, activation = 128, 'Swish'
    hidden_dim = [1024]
    data_noise_std, sgld_noise_std, sgld_lr, sgld_num_steps = 3e-2, 7.5e-3, 2.0, 60
    
elif dataset == 'mnist' or dataset == 'fashionmnist':
    channels, kernels, strides, paddings =[64,64,32,32], [3,4,4,4], [1,2,2,2], [1,1,1,1]
    latent_dim, activation = 128, 'Swish'
    data_noise_std, sgld_noise_std, sgld_lr, sgld_num_steps =  3e-2, 7.5e-3, 2.0, 60
    hidden_dim = [128]
else:
    raise NotImplementError
# leak, batchnorm, dropout = 0.2, False, None

# buffer_init, buffer_dup_allowed = True, True
data_dir = '../../../sebm_data/'
load_version = 'cebm_%sss-out=-1-d=%s-seed=%d-lr=%s-zd=%d-d_ns=%s-sgld-ns=%s-lr=%s-steps=%s-reg=%s-act=%s-arch=%s' % (ss, dataset, seed, lr, latent_dim, data_noise_std, sgld_noise_std, sgld_lr, sgld_num_steps,  reg_alpha, activation, arch)
if arch == 'wresnet':
    ebm = init_cebm(arch=arch,
                    ss=ss,
                    optimize_priors=optimize_priors,
                    device=device,
                    depth=depth,
                    width=width,
                    hidden_dim=hidden_dim,
                    latent_dim=latent_dim,
                    act=activation)
elif arch == 'simplenet2':
    ebm = init_cebm(arch=arch,
                    ss=ss,
                    latent_dim=latent_dim,
                    optimize_priors=optimize_priors,
                    device=device,
                    im_height=im_height, 
                    im_width=im_width, 
                    input_channels=input_channels, 
                    channels=channels, 
                    kernels=kernels, 
                    strides=strides, 
                    paddings=paddings, 
                    hidden_dim=hidden_dim,
                    activation=activation)
elif arch == 'simplenet5':
    ebm = init_cebm(arch=arch,
                    ss=ss,
                    latent_dim=latent_dim,
                    optimize_priors=optimize_priors,
                    device=device,
                    im_height=im_height, 
                    im_width=im_width, 
                    input_channels=input_channels, 
                    channels=channels, 
                    kernels=kernels, 
                    strides=strides, 
                    paddings=paddings, 
                    activation=activation,
                    leak=leak,
                    last_act=False,
                    batchnorm=batchnorm,
                    dropout=dropout)
        
ebm = ebm.cuda().to(device)
print('Loading trained weights..')
ebm.load_state_dict(torch.load('../weights/final/cp-%s' % load_version)['model_state_dict'])

In [ ]:
from sebm.eval import *
evaluator = Evaluator_EBM(ebm, device, dataset, data_dir, data_noise_std=1e-2)

In [ ]:
# images_ebm = evaluator.uncond_sampling(batch_size=100, 
#                                           sgld_steps=1000,
#                                           sgld_lr=2.0,
#                                           sgld_noise_std=1e-3,
#                                           grad_clipping=False,
#                                           init_samples=None,
#                                           logging_interval=None)
# evaluator.plot_final_samples(images_ebm, fs=0.5, save=False)

In [ ]:
fewshots(model_name='cebm', evaluator=evaluator)

In [ ]:
# similarity_z_space(evaluator, train_batch_size=2000, test_batch_size=2000, model_name='cebm')

In [ ]:
# metrics =  ['vae_z', 'cebm_z']
# datasets = ['cifar10']
# import torch
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_palette('colorblind')

# # for d in datasets:
# #     for m in metrics:
# plot_confusion_matrices(datasets, metrics, fs=6)

In [ ]:
# from sebm.eval import *
# train_batch_size = 1000
# data_dir = '/home/hao/Research/sebm_data/'
# # dataset = 'svhn'
# import torch
# for i in range(3):
#     print('processing on dataset %d' % (i+1))
#     test_data = torch.load('/home/hao/Research/sebm_data/overviewfig/%s_test/%d.pt' % (dataset, i+1))
#     min_distances, min_labels, nns = similarity_z_space_fewshots(evaluator, train_batch_size, test_data)
#     plot_nearest_neighbors(test_data, nns, min_labels, min_distances, fs=1, save_name='%s_cebm_%d' % (dataset, (i+1)))

In [ ]:
# scores = evaluator.oodauc(dataset_ood='emnist', score='gradient')
# scores

In [ ]:
# from sebm.eval import draw_one_batch
# dataset = 'cifar10'
# list_num_shots = [10]
# for num_shots in list_num_shots:
#     print('num of shots=%d' % num_shots)
#     for i in range(10):
#         images, labels = draw_one_batch(num_shots, 
#                                          dataset, 
#                                          data_dir, 
#                                          train=False, 
#                                          normalize=False, 
#                                          flatten=False)
#         data = {'images': images, 'labels' : labels}
#         torch.save(data, '/home/hao/Research/sebm_data/overviewfig/%s_test//%d.pt' % (dataset, i+1))

In [ ]:
# logging_interval = 200 # None otherwise
# test_batch_size = 10
# init_samples, labels = draw_one_batch(1, dataset, data_dir, train=False, normalize=True, flatten=False)

# images_ebm = evaluator.uncond_sampling_ll(sample_size=3,
#                                           batch_size=test_batch_size, 
#                                           sgld_steps=1000,
#                                           sgld_lr=2.0,
#                                           sgld_noise_std=1e-3,
#                                           init_images=init_samples,
#                                           grad_clipping=False,
#                                           logging_interval=None)
# # nearestneighbours = evaluator.nn_latents(images_ebm)
# # plot_evolving_samples(images_ebm, nearestneighbours, fs=8, save_name=None)

In [ ]:
# plot_likelihood_cond_samples(images_ebm, fs=1)

In [ ]:
# evaluator.plot_all_samples(list_images=images_ebm, fs=1.0)

In [ ]:
# evaluator.plot_all_samples(images_ebm, fs=6, save=True)

In [ ]:
# train_logistic_classifier(evaluator)

In [ ]:
from sebm.util import cnn_output_shape
cnn_output_shape(h=28,w=28,kernels=[4,4,4,4],strides=[2,2,2,2],paddings=[1,1,1,1])

In [ ]:
4*512